# ST7 Planification quotidienne d’une équipe mobile
# Phase I

## Importation de modules

In [1]:
# module importation
import numpy as np
from math import ceil
import matplotlib.pyplot as plt
from gurobipy import *
% matplotlib inline

# utilities
from utils import *
from copy import deepcopy

# model classes for employees and nodes
from models_v2 import Employee, Node, Task, Home, Unavail

Déclaration des constantes et des indices :

In [ ]:
W = U = T = V = 0
employees = homes = tasks = unavails = nodes = []

Fonction de lecture de données

In [2]:
def load_data_from_path(path_to_instance: str):
    # load employee data
    Employee.load_excel(path_to_instance)

    # load node data
    Node.clear_previous_data()
    for cls in [Home, Task, Unavail]:
        cls.load_excel(path_to_instance)
    Node.initialize_distance()

    # constants
    global W, U, T, V
    (W, U, T, V) = (Task.count, Unavail.count, Employee.count, Node.count)

    # indices of employees, homes, lunches, tasks, unavailabilities
    global employees, homes, tasks, unavails, nodes
    employees = list(range(T))
    homes = list(range(T))
    tasks = list(range(T, T + W))
    unavails = list((range(T + W, V)))
    nodes = list(range(V))

Lecture de données de test

In [3]:
path_to_test = path_bordeaux_v2 = "./data/InstancesV2/InstanceBordeauxV2.xlsx"
load_data_from_path(path_to_test)

La fonction suivante affiche un message d’erreur en orange

In [ ]:
class Solution:

    def __init__(self):
        self.x = np.zeros((V, V), dtype=np.intc)
        self.y = np.zeros((T, W), dtype=np.intc)
        self.b = np.zeros(W, dtype=np.intc)
        self.l = np.zeros((T, V), dtype=np.intc)

    def copy(self):
        """Return a deep copy of the instance"""
        return deepcopy(self)

    def validate_binary_vars(self):
        """Verify that x, y and l only contain binary variables"""
        def is_binary(arr: np.array):
            return np.min(arr) >= 0 and np.max(arr) <= 1

        to_verify = [(self.x, "x"), (self.y, "y"), (self.l, "l")]
        for arr, name in to_verify:
            if is_binary(arr):
                continue
            print_warning(f"{name} should contain binary variables, values other than 0 and 1 detected.")
            return False
        return True

    def validate(self):
        """
        validate the modifications
        :return: whether the constraints are verified
        """

        # verify that x, y and l contain only 0 and 1
        if not self.validate_binary_vars(): return False


        entrance_count = self.x.sum(axis=0)
        exit_count = self.x.sum(axis=1)
        # C1
        if not np.all(entrance_count == exit_count):
            print_warning("C1: The number of entrance of each node should be equal to the number of exit.")
            node_idx = np.argwhere(entrance_count != exit_count)[0]
            print_warning(f"Condition violated by node {node_idx} with {entrance_count[node_idx]} entrance(s) and {exit_count[node_idx]} exit(s).")
            return False



        # C2
        less_than_one_entrance = entrance_count <= 1
        if not np.all(less_than_one_entrance[tasks + homes]):
            print_warning("C2: Each task should be done by at most one employee.")
            for i in tasks:
                if not less_than_one_entrance[i]:
                    print_warning(f"Condition violated by node {i} done by {entrance_count[i]} employees.", "cyan")
            return False

        # C3.a
        exactly_one_entrance = entrance_count == 1
        if not np.all(exactly_one_entrance[unavails]):
            print_warning("C3.a: Unavailabilities should be visited.")
            unavail_indices = np.argwhere(~exactly_one_entrance)
            idx = unavail_indices[0]
            print_warning(f"Condition violated by node {idx}", "cyan")
            return False

        # C3.b
        for k in employees:
            if self.x[k, Employee.list[k].home()]:
                continue
            print_warning(f"C3.b: Each employee should visits their home.")
            print_warning(f"Condition violated by employee {k}.", "cyan")
            
            return False

        for idx in unavails:
            employee = Node.list[idx].employee
            employee_idx = Employee.index_of(employee)
            if self.x[employee_idx, idx] == 1:
                continue
            print_warning(f"C3.b: Each employee should visit his unavailabilities.")
            print_warning(f"Condition violated by employee {employee_idx}.","cyan")
            return False

        # C4
        for i in nodes:
            for j in nodes:
                if i == j or not self.x[i, j]: continue
                if self.y[:, i] == self.y[:, j]: continue
                print_warning(f"C4: Two consecutive tasks should be done by the same employee")
                print_warning(f"Condition violated by task {i} and {j}", "cyan")
                return False

        # C5
        for i in tasks:
            task: Task = Node.list[i]
            opening, closing = task.opening_time, task.closing_time
            if self.b[i] >= opening and self.b[i] + task.duration <= closing: continue
            print_warning(f"C5: A task should be worked on between its opening time and closing time")
            print_warning(f"Condition violated by task {i}", "cyan")
            return False

        # C6.a
        for k in employees:
            employee: Employee = Employee.list[k]
            employee_start = employee.start_time
            if employee_start == self.b[k]: continue
            print_warning(f"C6.a: Employees' start times should be respected")
            print_warning(f"Condition violated by employee {k}", "cyan")
            return False

        # C6.b
        for i in unavails:
            if self.b[i] == Node.list[i].opening_time: continue
            print_warning(f"C6.b: The beginning and end of unavailabilities should be respected")
            print_warning(f"Condition violated by Node {i}", "cyan")
            return False

        # C7
        # TODO: verify C7

        # C8
        # TODO: verify C8

        # C9
        for k in employees:
            for i in tasks:
                if not self.x[i, k]: continue
                task: Task = Node.list[i]
                employee: Employee = Employee.list[k]
                if self.b[k] + task.duration + (Node.distance[i, k] / Employee.speed) + self.l[
                    k, i] * 60 <= employee.end_time: continue
                print_warning("C9: An employee should have enough time to go back home.")
                print_warning(f"Condition violated by employee {k} after task {i}", "cyan")
                return False

        # C10.a
        for k in employees:
            if sum(self.l[k, :]) == 1: continue
            print_warning("C10.a: Each employee has one unique lunch")
            print_warning(f"Condition violated by employee {k}", "cyan")
            return False

        # C10.b
        for k in employees:
            for i in nodes:
                if not self.l[k, i]: continue
                if self.y[k, i]: continue
                print_warning(f"C10.b: Constraint violated")
                print_warning(f"Condition violated by employee {k} after lunch at node {i}")
                return False

        # C10.c
        for i in nodes:
            for k in employees:
                if not self.l[k, i]: continue
                task_duration = Node.list[i].duration
                if self.b[i] + task_duration + self.l[k, i] * 60 <= 14 * 60 and self.b[
                    i] + task_duration >= 12 * 60: continue
                print_warning("C10.c A task should be taken between 12AM and 2PM")
                print_warning(f"Condition violated by employee {k} after node {i}", "cyan")
                return False

        # C11
        for i in tasks + unavails:
            if sum(self.y[:, i]) <= sum(self.x[:, i]): continue
            print_warning("C11: This constraint is violated")
            print_warning(f"Condition violated by node {i}", "cyan")
            return False

        # C12
        for i in tasks + unavails:
            for j in tasks + unavails:
                if i == j or self.x[i, j]: continue
                if self.b[i] + Node.list[i].duration + (Node.distance[i, j] / Employee.speed) + sum(
                        self.l[:, i]) * 60 <= self.b[j]: continue
                print_warning("C12: The traveling time between two nodes should be sufficient.")
                print_warning(f"Condition violated between node {i} and {j}", "cyan")
                return False

        # C13
        for k in employees:
            for i in tasks:
                if not self.y[k, i]: continue
                employee: Employee = Employee.list[k]
                task: Task = Node.list[i]
                if employee.level >= task.level: continue
                print_warning("C13: An employee can only do tasks that they are able to do.")
                print_warning(f"Condition violated by employee {k} at node {i}", "cyan")
                print_warning(repr(employee), "green")
                print_warning(repr(task), "green")
                return False

        # Every condition is verified
        return True